![](https://raw.githubusercontent.com/chenkenanalytic/img/master/af/aifreeteam.png)
# Ch 02 - Unit 07：【實作】讓 AI 具備寫文字的能力：GPT-2 周意解夢

## Step 1: 安裝相關套件




In [2]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Step 2: 準備AI模型

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# 設定 tokenizer 的 pad_token
tokenizer.pad_token = tokenizer.eos_token
# model.config.pad_token_id = tokenizer.pad_token_id

In [5]:
import torch

# 設置模型到 GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

def generate_text(prompt, max_length=100):
    model.eval()
    # 編碼輸入文本，並同時生成 attention mask
    encoded_input = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    with torch.no_grad():
        # 傳遞 attention_mask 到生成函數
        output = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# 測試生成的文本
print(generate_text("夢見大火"))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


夢見大火神・アグニ 獄炎龍・インフェルノ 狂犬・相良猛 盗賊の極意, Chrollo Lucilfer 竹取の翠月花・かぐや姫 火の幻蛇龍の渦龍喚士・ヴィゴ 猫塚かり


## Step 3: 準備訓練資料集

In [8]:
import pandas as pd

# 創建數據字典
data = {
    "text": [
        "夢見大水 - 水象徵財富和機遇，夢見大水預示著可能會有一筆意外之財或是新的機遇即將來臨。",
        "夢見火災 - 火通常代表破壞性的力量或情感的爆發，此夢可能提醒你需要控制情緒或避免危險。",
        "夢見牙齒掉落 - 在周易中，牙齒象徵穩固和長壽，牙齒掉落可能象徵著對家庭或健康的擔憂。",
        "夢見飛翔 - 飛翔在夢中通常象徵自由和逃脫現實的束縛，表明你渴望更自由的生活方式或思想。",
        "夢見考試 - 考試夢往往反映了對即將到來的挑戰或評估的焦慮，可能是生活中某個領域的壓力。",
        "夢見赤裸 - 赤裸通常象徵著脆弱或恐懼被人了解真實的自我，可能表明你在某些方面感到不安全。",
        "夢見迷路 - 迷路反映了在生活中感到迷茫或目標不明確，可能需要重新考慮自己的方向。",
        "夢見被追逐 - 被追逐通常象徵著逃避問題，建議面對生活中的困難而不是逃避。",
        "夢見花開 - 花開象徵著新的開始和希望，夢見花開預示著好事即將發生。",
        "夢見掉入水中 - 掉入水中可能象徵著情感的淹沒或不安，建議你關注自己的情感狀態並尋求支持。"
    ]
}

# 轉換為 DataFrame
df = pd.DataFrame(data)

# 保存為 CSV 檔案
df.to_csv('dreams_dataset.csv', index=False)

## Step 4: 文本資料前處理

In [9]:
# 加載 CSV 檔案
df = pd.read_csv('dreams_dataset.csv')

# 分割訓練集和測試集
train_dataset = df.sample(frac=0.8, random_state=200)  # 例如，80% 數據作為訓練集
test_dataset = df.drop(train_dataset.index)


# Tokenizer 處理（確保已經加載了 tokenizer）
train_encodings = tokenizer(train_dataset['text'].tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_dataset['text'].tolist(), truncation=True, padding=True, max_length=512)

## Step 5：訓練模型（微調）

In [10]:
from torch.utils.data import Dataset, DataLoader
import torch

# 定義一個 Dataset 子類
class DreamDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

# 創建 DataLoader
train_loader = DataLoader(DreamDataset(train_encodings), batch_size=4, shuffle=True)

In [11]:
# 設置模型到 GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# 定義訓練函數
def train(epoch):
    model.train()
    for _, data in enumerate(train_loader):
        inputs, labels = data['input_ids'].to(device), data['input_ids'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

In [12]:
from tqdm import tqdm

for epoch in tqdm(range(100)):  # 訓練 100 個 Epoch
    train(epoch)

  1%|          | 1/100 [00:00<01:13,  1.34it/s]

Epoch: 0, Loss: 4.705190181732178
Epoch: 0, Loss: 4.70341157913208


  2%|▏         | 2/100 [00:01<00:48,  2.02it/s]

Epoch: 1, Loss: 3.677560806274414
Epoch: 1, Loss: 3.4290151596069336


  3%|▎         | 3/100 [00:01<00:39,  2.45it/s]

...

Epoch: 97, Loss: 0.09177032858133316
Epoch: 97, Loss: 0.06578346341848373


 99%|█████████▉| 99/100 [00:31<00:00,  3.16it/s]

Epoch: 98, Loss: 0.037548888474702835
Epoch: 98, Loss: 0.0670745000243187


100%|██████████| 100/100 [00:31<00:00,  3.17it/s]

Epoch: 99, Loss: 0.02861410565674305
Epoch: 99, Loss: 0.06084311008453369


In [13]:
# 測試生成的文本
print(generate_text("我夢見自己全身赤落的站在黑暗當中"))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


我夢見自己全身赤落的站在黑暗當中，可能需要控制情緒或避免危險。
